### Tanks 4.09d Vertical Fixed Roof Tank Calculations 

In [1]:
import math
import pandas as pd
import numpy as np

In [22]:
class VerticalFixedRoofTank:
    
    def __init__(self, 
                 tkshellht, 
                 skliqht, 
                 tkrfslope, 
                 diameter,
                 ins, 
                 solarabs, 
                 tax, 
                 tan, 
                 atm_p_local,
                throughput,
                product_factor):
        
        '''
        tax: Average Daily Maximum Ambient Temperature, user defined, in deg F.
        tan: Average Daily Minimum Ambient Temperature, user defined, in deg F.
        solarabs: Tank surface solar absorptance, user defined, depends on tank color.
        ins: Average daily total insulation, defined to be 1,491 (btu/ft^2*d) (from Table 7.1-7)
        '''
        self.tkshellht = tkshellht
        self.skliqht = skliqht
        self.tkrfslope = tkrfslope
        self.diameter = diameter
        self.ins = ins
        self.solarabs = solarabs
        self.throughput = throughput
        self.product_factor = product_factor
        
        #Values are in deg. F
        self.tax = tax
        self.tan = tan
        
        self.rad_ = (1/2.)*self.diameter
        self.hro_ = (1/3.)*self.rad_*self.tkrfslope
        
        self.hvo = self.tkshellht - self.skliqht + self.hro_
        
        #Breather vent pressure setting (p_bp: pressure setting; p_bv: vacuum setting)
        self.p_bp = 0.03
        self.p_bv = -0.03
        
        #Returns values in def. Rankine
        self.tax_r = self.tax + 459.7
        self.tan_r = self.tan + 459.7
        
        self.delta_ta_r = self.tax_r - self.tan_r
        
        self.taa = (self.tax_r + self.tan_r) * (1/2.)
        self.tb = self.taa + (0.003*self.solarabs*self.ins)
        
        #Atmospheric pressure at facility, user defined from Table 7.1-7.
        self.atm_p_local = atm_p_local
        
    #TODO: Calculate Turnover Factor (KN)
    
    def kp(self):
        if self.product_factor == 'crude oils':
            return 0.75
        elif self.product_factor == 'other stocks':
            return 1.0
        #TODO: Raise error if something else is entered.
    
    '''
    See note from Eqn: 1-35, throughput is in gal and 
    converted to BBL (1 BBL = 42 Gal)
    '''
    def vq(self):
        return 5.614*(self.throughput)*(1/42.)
        
    def hvo(self):
        '''
        Returns the Vapor Space Outage in units of Feet as calculated from Eqn: 1-16
        '''
        return self.hvo
        
    def vv(self):
        '''
        Returns the Vapor Space Volume of the storage tank.
        '''
        return (math.pi)*(1/4.)*(self.diameter**2)*self.hvo
    
    def tla(self):
        '''
        Returns the Daily Average Liquid Surface Temperature, as caluclated from Eqn: 1-28
        '''

        return (0.4*self.taa) + (0.6*self.tb) + ((0.005*self.solarabs)*(self.ins))
    
    def tv(self):
        '''
        Returns Average Vapor Temperature, in Rankine as caluclated from Eqn: 1-33
        '''
        return (0.7*self.taa) + (0.3*self.tb) + (0.009*self.solarabs*self.ins)
    
    def deltv(self):
        '''
        Returns the Daily Average Temperature Range, in Rankine as calculated from Eqn: 1-7
        '''
        return (0.7*self.delta_ta_r) + (0.02*self.solarabs*self.ins)
    
    def tlx_r(self):
        '''
        Returns the Maximum Liquid Temperature, in Rankine as calculated from Figure 7.1-17
        '''
        return ((0.4*self.taa) + (0.6*self.tb) + ((0.005*self.solarabs)*(self.ins))) + (0.25*((0.7*self.delta_ta_r) + (0.02*self.solarabs*self.ins)))
    
    def tln_r(self):
        '''
        Returns the Minimum Liquid Temperature, in Rankine as calculated from Figure 7.1-17
        '''
        return ((0.4*self.taa) + (0.6*self.tb) + ((0.005*self.solarabs)*(self.ins))) - (0.25*((0.7*self.delta_ta_r) + (0.02*self.solarabs*self.ins)))
    
    def tlx_f(self):
        '''
        Returns the Maximum Liquid Temperature, in F as calculated from Figure 7.1-17
        Assume: 1 R − 459.67 = -458.7 F
        '''
        return (((0.4*self.taa) + (0.6*self.tb) + ((0.005*self.solarabs)*(self.ins))) + (0.25*((0.7*self.delta_ta_r) + (0.02*self.solarabs*self.ins)))) - 458.67
    
    def tln_f(self):
        '''
        Returns the Minimum Liquid Temperature, in F as calculated from Figure 7.1-17
        Assume: 1 R − 459.67 = -458.7 F
        '''
        return (((0.4*self.taa) + (0.6*self.tb) + ((0.005*self.solarabs)*(self.ins))) - (0.25*((0.7*self.delta_ta_r) + (0.02*self.solarabs*self.ins)))) - 458.67
    
    def bventpress(self):
        '''
        Returns Breather Vent Pressure, delta_pb, calculated from Eqn: 1-10
        '''
        return self.p_bp - self.p_bv

In [25]:
tank = VerticalFixedRoofTank(tkshellht=12, skliqht=8, 
                             tkrfslope=0.0625, diameter=6,
                             ins=1491, solarabs=0.25, 
                             tax=63.5, tan=37.9, atm_p_local=12.08, 
                             throughput=8450, product_factor='crude oils')

In [26]:
tank.kp()

0.75

In [4]:
tank.vq()

1129.4833333333331

In [ ]:
vv = tank.vv()
vv

In [ ]:
tank.tla()

In [ ]:
tank.tv()

In [ ]:
tank.deltv()

In [ ]:
tank.tlx_r()

In [ ]:
tank.tln_r()

In [ ]:
tank.tlx_f()

In [ ]:
tank.tln_f()

In [ ]:
tank.bventpress()

In [ ]:
cyclohexane_A = 6.845
cyclohexane_B = 1203.5
cyclohexane_C = 222.86

toluene_A = 7.017
toluene_B = 1377.6
toluene_C = 222.64

benzene_A = 6.906
benzene_B = 1211.0
benzene_C = 220.79

#Converts from Rankine to deg C.
tla_c = (tank.tla() - 491.7) * (5/9.)
tla_f = (tla_c * (9/5.)) + 32
tla_f

In [ ]:
tla_c

In [ ]:
def calVaporPressure(an_a, an_b, an_c, tla_c):
    '''
    Returns a list of the calculated vapor pressures based on the Antone coefficients. 
    Element 0 of the list of the value in PSIA and Element 1 of the list is the value in mmHg.
    tla_c: Daily Average Liquid Surface Temperature in deg C.
    '''
    return [10**(an_a - ((an_b) / (tla_c + an_c)))/ 51.715, 10**(an_a - ((an_b) / (tla_c + an_c)))]

In [ ]:
phMe_vp = calVaporPressure(an_a=toluene_A, an_b=toluene_B, an_c=toluene_C, tla_c=tla_c)
phH_vp = calVaporPressure(an_a=benzene_A, an_b=benzene_B, an_c=benzene_C, tla_c=tla_c)
cyH_vp = calVaporPressure(an_a=cyclohexane_A, an_b=cyclohexane_B, an_c=cyclohexane_C, tla_c=tla_c)

In [ ]:
phMe_vp

In [ ]:
phH_vp

In [ ]:
cyH_vp

In [ ]:
comp_amt = [2812.0, 258, 101]
comp_mw = [78.11, 92.14, 84.16]
comp_name = ['Benzene', 'Toluene', 'Cyclohexane']
comp_vp = [phH_vp[0], phMe_vp[0], cyH_vp[0]]

df = pd.DataFrame({'component': comp_name, 
                   'amount_lb': comp_amt, 
                   'mw_lbs_mol': comp_mw, 
                   'comp_vp': comp_vp})

df['comp_mole'] = df['amount_lb'] / df['mw_lbs_mol']
tot_moles = np.sum(df['comp_mole'].tolist())

df['comp_mole_xi'] = df['comp_mole'] / tot_moles
df['comp_partial'] = df['comp_mole_xi'] * df['comp_vp']
vp_mixture = np.sum(df['comp_partial'].tolist())

df['comp_vap_mole_frac'] = df['comp_partial'] / vp_mixture
df['comp_vapor_mw_xi'] = df['mw_lbs_mol'] * df['comp_vap_mole_frac']
vapor_mw = np.sum(df['comp_vapor_mw_xi'].tolist())

In [ ]:
vapor_mw

In [ ]:
vp_mixture

In [ ]:
def stockDensity(mv, pva, tv):
    '''
    Uses R = 10.731 psia*ft3 / lb-mole* deg R
    Returns stock density in units of lbs/ft3
    '''
    return ((mv*pva) / (10.731 *tv))

In [ ]:
wv = stockDensity(mv=vapor_mw, pva=vp_mixture, tv=tank.tv())
wv

In [ ]:
#TODO: Calculate vapor pressure range.

def vapPressureRange(plx, pln):
    '''
    Returns the vapor pressure range, in PSIA, as calculated from Eqn: 1-10.
    '''
    return plx - pln

In [ ]:
#Converts from Rankine to deg C.
tlx_c = (tank.tlx_r() - 491.7) * (5/9.)
tln_c = (tank.tln_r() - 491.7) * (5/9.)

In [ ]:
tlx_c

In [ ]:
tln_c

In [ ]:
phMe_vp_tlx = calVaporPressure(an_a=toluene_A, an_b=toluene_B, an_c=toluene_C, tla_c=tlx_c)
phH_vp_tlx = calVaporPressure(an_a=benzene_A, an_b=benzene_B, an_c=benzene_C, tla_c=tlx_c)
cyH_vp_tlx = calVaporPressure(an_a=cyclohexane_A, an_b=cyclohexane_B, an_c=cyclohexane_C, tla_c=tlx_c)

In [ ]:
phMe_vp_tln = calVaporPressure(an_a=toluene_A, an_b=toluene_B, an_c=toluene_C, tla_c=tln_c)
phH_vp_tln = calVaporPressure(an_a=benzene_A, an_b=benzene_B, an_c=benzene_C, tla_c=tln_c)
cyH_vp_tln = calVaporPressure(an_a=cyclohexane_A, an_b=cyclohexane_B, an_c=cyclohexane_C, tla_c=tln_c)

In [ ]:
comp_vp_tln = [phH_vp_tln[0], phMe_vp_tln[0], cyH_vp_tln[0]]
comp_vp_tlx = [phH_vp_tlx[0], phMe_vp_tlx[0], cyH_vp_tlx[0]]

df['comp_vp_tln'] = comp_vp_tln
df['comp_vp_tlx'] = comp_vp_tlx

In [ ]:
df['comp_partial_tln'] = df['comp_mole_xi'] * df['comp_vp_tln']
df['comp_partial_tlx'] = df['comp_mole_xi'] * df['comp_vp_tlx']
df

In [ ]:
tot_vp_tln = np.sum(df['comp_partial_tln'].tolist())
tot_vp_tlx = np.sum(df['comp_partial_tlx'].tolist())

In [ ]:
delpv = vapPressureRange(plx=tot_vp_tlx, pln=tot_vp_tln)

In [ ]:
def vaporSpaceExpansionFactor(deltv, tla, delpv, delbpv, atmp, pva):
    '''
    Returns the Vapor Space Expansion Factor (ke), as calculated from Eqn: 1-5.
    '''
    return (deltv/tla) + ((delpv - delbpv) / (atmp - pva))
ke = vaporSpaceExpansionFactor(deltv=tank.deltv(), 
                          tla=tank.tla(), 
                          delpv=delpv, 
                          delbpv=tank.bventpress(), 
                          atmp=12.08, 
                          pva=vp_mixture)
ke

In [ ]:
def ventedVaporSpaceSatFactor(pva, hvo):
    return 1/(1 + (0.053*pva*hvo))

In [ ]:
ks = ventedVaporSpaceSatFactor(pva=vp_mixture, hvo=tank.hvo)

In [ ]:
def calculateStandingLosses(vv, wv, ke, ks):
    '''
    Returns the standing losses from the tank in lbs/year.
    '''
    return 365*vv*wv*ke*ks

In [ ]:
calculateStandingLosses(vv=vv, wv=wv, ke=ke, ks=ks)

In [ ]:
def calculateWorkingLosses(vq, kn, kp, wv, kb):
    '''
    Returns the working losses from the tank in lbs/year.
    '''
    return vq*kn*kp*wv*kb